In [113]:
import pandas as pd
import numpy as np
import ast
from sklearn.metrics.pairwise import cosine_similarity

In [114]:
recipes = pd.read_csv('/content/RAW_recipes.csv')
recipes.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [115]:
recipes.shape

(231637, 12)

In [116]:
recipes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231637 entries, 0 to 231636
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   name            231636 non-null  object
 1   id              231637 non-null  int64 
 2   minutes         231637 non-null  int64 
 3   contributor_id  231637 non-null  int64 
 4   submitted       231637 non-null  object
 5   tags            231637 non-null  object
 6   nutrition       231637 non-null  object
 7   n_steps         231637 non-null  int64 
 8   steps           231637 non-null  object
 9   description     226658 non-null  object
 10  ingredients     231637 non-null  object
 11  n_ingredients   231637 non-null  int64 
dtypes: int64(5), object(7)
memory usage: 21.2+ MB


In [117]:
recipes.describe()

,id,minutes,contributor_id,n_steps,n_ingredients
count,231637.000000,2.316370e+05,2.316370e+05,231637.000000,231637.000000
mean,222014.708984,9.398546e+03,5.534885e+06,9.765499,9.051153
std,141206.635626,4.461963e+06,9.979141e+07,5.995128,3.734796
min,38.000000,0.000000e+00,2.700000e+01,0.000000,1.000000
25%,99944.000000,2.000000e+01,5.690500e+04,6.000000,6.000000
50%,207249.000000,4.000000e+01,1.736140e+05,9.000000,9.000000
75%,333816.000000,6.500000e+01,3.982750e+05,12.000000,11.000000
max,537716.000000,2.147484e+09,2.002290e+09,145.000000,43.000000


In [118]:
recipes.isnull().sum()

,0
name,1
id,0
minutes,0
contributor_id,0
submitted,0
tags,0
nutrition,0
n_steps,0
steps,0
description,4979


In [119]:
recipes = recipes.dropna()
recipes.isnull().sum()

,0
name,0
id,0
minutes,0
contributor_id,0
submitted,0
tags,0
nutrition,0
n_steps,0
steps,0
description,0


In [120]:
recipes.shape

(226657, 12)

In [121]:
#Dividing the tags in each entry
recipes['tags'] = recipes['tags'].apply(lambda x: ast.literal_eval(x))
tags = recipes['tags'].explode()
tags

,tags
0,60-minutes-or-less
0,time-to-make
0,course
0,main-ingredient
0,cuisine
...,...
231636,dietary
231636,comfort-food
231636,taste-mood
231636,sweet


In [122]:
len(tags.unique())

552

In [123]:
tag_counts = tags.value_counts()
tags_filtered = tag_counts[tag_counts > 10000]
tags_filtered

,count
tags,
preparation,225568
time-to-make,220353
course,213602
main-ingredient,166456
dietary,160444
...,...
beverages,10913
sweet,10503
savory,10428


In [124]:
len(tags_filtered)

80

In [125]:
recipes['nutrition'] = recipes['nutrition'].apply(lambda x: ast.literal_eval(x))
recipes[['calories', 'fat', 'sugar', 'sodium', 'protein', 'saturated_fat', 'carbohydrates']] = recipes['nutrition'].apply(pd.Series)
recipes = recipes.drop(columns=['nutrition'])
recipes.head()

,name,id,minutes,contributor_id,submitted,tags,n_steps,steps,description,ingredients,n_ingredients,calories,fat,sugar,sodium,protein,saturated_fat,carbohydrates
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"[60-minutes-or-less, time-to-make, course, mai...",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,51.5,0.0,13.0,0.0,2.0,0.0,4.0
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"[30-minutes-or-less, time-to-make, course, mai...",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,173.4,18.0,0.0,17.0,22.0,35.0,1.0
2,all in the kitchen chili,112140,130,196586,2005-02-25,"[time-to-make, course, preparation, main-dish,...",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,269.8,22.0,32.0,48.0,39.0,27.0,5.0
3,alouette potatoes,59389,45,68585,2003-04-14,"[60-minutes-or-less, time-to-make, course, mai...",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,368.1,17.0,10.0,2.0,14.0,8.0,20.0
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"[weeknight, time-to-make, course, main-ingredi...",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,352.9,1.0,337.0,23.0,3.0,0.0,28.0


In [126]:
recipes['steps'] = recipes['steps'].apply(lambda x: ast.literal_eval(x))
recipes['steps'][0]

['make a choice and proceed with recipe',
 'depending on size of squash , cut into half or fourths',
 'remove seeds',
 'for spicy squash , drizzle olive oil or melted butter over each cut squash piece',
 'season with mexican seasoning mix ii',
 'for sweet squash , drizzle melted honey , butter , grated piloncillo over each cut squash piece',
 'season with sweet mexican spice mix',
 'bake at 350 degrees , again depending on size , for 40 minutes up to an hour , until a fork can easily pierce the skin',
 'be careful not to burn the squash especially if you opt to use sugar or butter',
 'if you feel more comfortable , cover the squash with aluminum foil the first half hour , give or take , of baking',
 'if desired , season with salt']

In [127]:
recipes['ingredients'] = recipes['ingredients'].apply(lambda x: ast.literal_eval(x))
recipes['ingredients'][0]

['winter squash',
 'mexican seasoning',
 'mixed spice',
 'honey',
 'butter',
 'olive oil',
 'salt']

In [128]:
ingredients = recipes['ingredients'].explode()
ing_counts = ingredients.value_counts()
ing_counts

,count
ingredients,
salt,83781
butter,53788
sugar,43419
onion,38168
water,34060
...,...
onion bread,1
nonfat berry yogurt,1
fat-free apricot yogurt,1


In [129]:
ing_filtered = ing_counts[ing_counts > 2000]
ing_filtered

,count
ingredients,
salt,83781
butter,53788
sugar,43419
onion,38168
water,34060
...,...
beef broth,2045
shallot,2029
"lemon, juice of",2019


In [130]:
for ing in list(ing_filtered.index):
  if 'chicken' in ing:
    print(ing)

chicken broth
chicken stock
boneless skinless chicken breasts
chicken breasts
chicken


In [131]:
def check_values(row,values):
  return pd.Series({value: value in row for value in values})

In [132]:
# New column for each ingredient in the filtered list
new_cols_ing = recipes['ingredients'].apply(check_values,values=list(ing_filtered.index))
new_cols_ing

,salt,butter,sugar,onion,water,eggs,olive oil,flour,garlic cloves,milk,...,cooking spray,skim milk,cream,chicken,warm water,beef broth,shallot,"lemon, juice of",ground coriander,fresh mushrooms
0,True,True,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,True,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
2,True,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,True,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231632,False,False,True,True,False,False,True,False,True,False,...,False,False,False,False,False,False,False,False,False,False
231633,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
231634,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
231635,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [133]:
# New column for each ingredient in the filtered list
new_col_tag = recipes['tags'].apply(check_values,values=list(tags_filtered.index))
new_col_tag

,preparation,time-to-make,course,main-ingredient,dietary,easy,occasion,cuisine,low-in-something,main-dish,...,free-of-something,condiments-etc,high-in-something,soups-stews,technique,beverages,sweet,savory,potluck,potatoes
0,True,True,True,True,True,True,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
1,True,True,True,True,True,True,True,True,False,True,...,False,False,False,False,False,False,False,False,False,False
2,True,True,True,False,True,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
3,True,True,True,True,True,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,True,True,True,True,True,False,True,True,False,False,...,False,True,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231632,True,True,True,True,False,True,False,True,False,False,...,False,False,False,True,False,False,False,False,False,False
231633,True,True,True,False,True,True,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
231634,True,True,True,True,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
231635,True,True,True,False,True,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False


In [134]:
# Combining tags and indredients together
tags_ings = pd.concat([new_cols_ing,new_col_tag],axis=1)
tags_ings.head()

,salt,butter,sugar,onion,water,eggs,olive oil,flour,garlic cloves,milk,...,free-of-something,condiments-etc,high-in-something,soups-stews,technique,beverages,sweet,savory,potluck,potatoes
0,True,True,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,True,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
2,True,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,True,False,True,False,False,False,False,False,False,False,...,False,True,False,False,True,False,False,False,False,False


In [135]:
recipes_final = recipes.drop(columns=['contributor_id', 'submitted', 'tags', 'steps','description','ingredients','name','n_steps','n_ingredients','minutes'])
recipes_final = pd.concat([recipes_final, tags_ings],axis=1)
recipes_final.head()

,id,calories,fat,sugar,sodium,protein,saturated_fat,carbohydrates,salt,butter,...,free-of-something,condiments-etc,high-in-something,soups-stews,technique,beverages,sweet,savory,potluck,potatoes
0,137739,51.5,0.0,13.0,0.0,2.0,0.0,4.0,True,True,...,False,False,False,False,False,False,False,False,False,False
1,31490,173.4,18.0,0.0,17.0,22.0,35.0,1.0,False,False,...,False,False,False,False,False,False,False,False,False,False
2,112140,269.8,22.0,32.0,48.0,39.0,27.0,5.0,True,False,...,False,False,False,False,False,False,False,False,False,False
3,59389,368.1,17.0,10.0,2.0,14.0,8.0,20.0,True,False,...,False,False,False,False,False,False,False,False,False,True
4,44061,352.9,1.0,337.0,23.0,3.0,0.0,28.0,True,False,...,False,True,False,False,True,False,False,False,False,False


In [136]:
dublicate_columns = recipes_final.columns[recipes_final.columns.duplicated()].tolist()
dublicate_columns

['sugar', 'chicken', 'cheese', 'potatoes']

In [137]:
recipes_final.columns.value_counts()

,count
cheese,2
chicken,2
sugar,2
potatoes,2
beef broth,1
...,...
ketchup,1
balsamic vinegar,1
buttermilk,1
cilantro,1


In [138]:
recipes_final = recipes_final.loc[:, ~recipes_final.columns.duplicated()]
recipes_final.columns.value_counts()

,count
potluck,1
salads,1
seafood,1
one-dish-meal,1
beef,1
...,...
protein,1
sodium,1
sugar,1
fat,1


In [139]:
recipes_final.to_parquet('/content/recipes_final.parquet')

In [140]:
recipes_ingtag = recipes.drop(columns=['name','steps','description','contributor_id','submitted','n_ingredients','n_steps','minutes','calories','fat','sodium','sugar','protein','saturated_fat','carbohydrates'])
recipes_ingtag.head()

,id,tags,ingredients
0,137739,"[60-minutes-or-less, time-to-make, course, mai...","[winter squash, mexican seasoning, mixed spice..."
1,31490,"[30-minutes-or-less, time-to-make, course, mai...","[prepared pizza crust, sausage patty, eggs, mi..."
2,112140,"[time-to-make, course, preparation, main-dish,...","[ground beef, yellow onions, diced tomatoes, t..."
3,59389,"[60-minutes-or-less, time-to-make, course, mai...","[spreadable cheese with garlic and herbs, new ..."
4,44061,"[weeknight, time-to-make, course, main-ingredi...","[tomato juice, apple cider vinegar, sugar, sal..."


In [141]:
recipes_ingtag.to_parquet('/content/recipes_ingtag.parquet')

In [142]:
recipes_steps = recipes.drop(columns=['contributor_id','submitted','n_ingredients','n_steps','minutes','calories','fat','sodium','sugar','protein','saturated_fat','carbohydrates','tags'])
recipes_steps.head()

,name,id,steps,description,ingredients
0,arriba baked winter squash mexican style,137739,"[make a choice and proceed with recipe, depend...",autumn is my favorite time of year to cook! th...,"[winter squash, mexican seasoning, mixed spice..."
1,a bit different breakfast pizza,31490,"[preheat oven to 425 degrees f, press dough in...",this recipe calls for the crust to be prebaked...,"[prepared pizza crust, sausage patty, eggs, mi..."
2,all in the kitchen chili,112140,"[brown ground beef in large pot, add chopped o...",this modified version of 'mom's' chili was a h...,"[ground beef, yellow onions, diced tomatoes, t..."
3,alouette potatoes,59389,[place potatoes in a large pot of lightly salt...,"this is a super easy, great tasting, make ahea...","[spreadable cheese with garlic and herbs, new ..."
4,amish tomato ketchup for canning,44061,"[mix all ingredients& boil for 2 1 / 2 hours ,...",my dh's amish mother raised him on this recipe...,"[tomato juice, apple cider vinegar, sugar, sal..."


In [143]:
recipes_steps.to_parquet('/content/recipes_steps.parquet')

TESTING

In [144]:
recipes_test = recipes_ingtag.copy()
recipes_test.head()

,id,tags,ingredients
0,137739,"[60-minutes-or-less, time-to-make, course, mai...","[winter squash, mexican seasoning, mixed spice..."
1,31490,"[30-minutes-or-less, time-to-make, course, mai...","[prepared pizza crust, sausage patty, eggs, mi..."
2,112140,"[time-to-make, course, preparation, main-dish,...","[ground beef, yellow onions, diced tomatoes, t..."
3,59389,"[60-minutes-or-less, time-to-make, course, mai...","[spreadable cheese with garlic and herbs, new ..."
4,44061,"[weeknight, time-to-make, course, main-ingredi...","[tomato juice, apple cider vinegar, sugar, sal..."


In [145]:
def all_tags_present(item_tags,selected):
  return all(string in item_tags for string in selected)

In [146]:
recipes_test['tag_match'] = recipes_test['tags'].apply(all_tags_present,selected = ['poultry','30-minutes-or-less'])
recipes_test = recipes_test[recipes_test['tag_match'] == True]

In [147]:
recipes_test.head()

,id,tags,ingredients,tag_match
57,32169,"[30-minutes-or-less, time-to-make, course, mai...","[tomatoes, garlic, onion, button mushrooms, ho...",True
75,42570,"[30-minutes-or-less, time-to-make, course, mai...","[boneless chicken breast, garlic, salt, cumin,...",True
233,219681,"[30-minutes-or-less, time-to-make, course, mai...","[chicken breasts, olive oil, lemon juice, whit...",True
336,399278,"[30-minutes-or-less, time-to-make, course, mai...","[cayenne pepper sauce, vegetable oil, granulat...",True
447,371549,"[30-minutes-or-less, time-to-make, course, mai...","[jalapeno peppers, fresh cilantro, light beer,...",True


In [148]:
ing_selected = ['chicken','lentils']

In [149]:
# Function to match all selected ingredients are found
def check_ingredients(row):
  ing_matched = row

  ingredients_str = ' '.join(str(ing).lower() for ing in ing_selected)

  for item in ing_selected:
    item = item.strip('s')
    if item.lower() not in ingredients_str:
      return False

  return True

In [150]:
recipes_test['ing_match'] = recipes_test['ingredients'].apply(check_ingredients)
recipes_test[recipes_test['ing_match'] == True]

/tmp/ipython-input-3082737126.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recipes_test['ing_match'] = recipes_test['ingredients'].apply(check_ingredients)


,id,tags,ingredients,tag_match,ing_match
57,32169,"[30-minutes-or-less, time-to-make, course, mai...","[tomatoes, garlic, onion, button mushrooms, ho...",True,True
75,42570,"[30-minutes-or-less, time-to-make, course, mai...","[boneless chicken breast, garlic, salt, cumin,...",True,True
233,219681,"[30-minutes-or-less, time-to-make, course, mai...","[chicken breasts, olive oil, lemon juice, whit...",True,True
336,399278,"[30-minutes-or-less, time-to-make, course, mai...","[cayenne pepper sauce, vegetable oil, granulat...",True,True
447,371549,"[30-minutes-or-less, time-to-make, course, mai...","[jalapeno peppers, fresh cilantro, light beer,...",True,True
...,...,...,...,...,...
230747,287586,"[30-minutes-or-less, time-to-make, course, mai...","[paprika, southwest seasoning, salt, pepper, b...",True,True
230755,281302,"[30-minutes-or-less, time-to-make, course, mai...","[chicken stock, rotel tomatoes, long grain ric...",True,True
231101,49767,"[30-minutes-or-less, time-to-make, course, mai...","[boneless skinless chicken breast halves, zucc...",True,True
231588,117910,"[30-minutes-or-less, time-to-make, course, mai...","[chicken broth, italian pastina, eggs, parmesa...",True,True


In [151]:
recipes_steps.head()

,name,id,steps,description,ingredients
0,arriba baked winter squash mexican style,137739,"[make a choice and proceed with recipe, depend...",autumn is my favorite time of year to cook! th...,"[winter squash, mexican seasoning, mixed spice..."
1,a bit different breakfast pizza,31490,"[preheat oven to 425 degrees f, press dough in...",this recipe calls for the crust to be prebaked...,"[prepared pizza crust, sausage patty, eggs, mi..."
2,all in the kitchen chili,112140,"[brown ground beef in large pot, add chopped o...",this modified version of 'mom's' chili was a h...,"[ground beef, yellow onions, diced tomatoes, t..."
3,alouette potatoes,59389,[place potatoes in a large pot of lightly salt...,"this is a super easy, great tasting, make ahea...","[spreadable cheese with garlic and herbs, new ..."
4,amish tomato ketchup for canning,44061,"[mix all ingredients& boil for 2 1 / 2 hours ,...",my dh's amish mother raised him on this recipe...,"[tomato juice, apple cider vinegar, sugar, sal..."


In [152]:
rec = recipes_steps[recipes_steps['id'] == 66735]
rec['steps'].values[0], rec['name'].values[0]

(['heat the oil in a non-stick saucepan over medium heat and cook the onion and garlic until softened',
  'add the lentils and stock and bring to a boil',
  'cover and cook for 10 minutes , or until lentils are tender',
  'stir through the lemon juice , parsley , salt and pepper',
  'while lentils are cooking , season the chicken with cumin',
  'heat a non-stick frying pan and spray with cooking oil spray',
  'cook chicken until cooked through',
  'serve over lentils',
  'lamb or fish fillets can be exchanged for chicken in this recipe'],
 'chicken with red lentils')

In [153]:
name = rec['name'].values[0].replace(' ','-')
name

'chicken-with-red-lentils'

In [154]:
link = f"https://www.food.com/recipe/{name}-{rec['id'].values[0]}"
link

'https://www.food.com/recipe/chicken-with-red-lentils-66735'

In [155]:
rec_final = recipes_final[recipes_final['id'] == 66735]
rec_final = rec_final.drop(columns=['id']).values.reshape(1,-1)

cosine_sim = cosine_similarity(rec_final,recipes_final.drop(columns=['id']))
sim_score = list(zip(recipes_final['id'],cosine_sim[0]))
sim_score = sorted(sim_score,key=lambda x:x[1],reverse=True)
sim_score = sim_score[1:6]
rec_indices = [i[0] for i in sim_score]
recipes = recipes_steps[recipes_steps['id'].isin(rec_indices)][['id','name','description']]
recipes


,id,name,description
44912,292894,chicken noodle goulash abs diet,my husband does the abs diet. this is one of t...
107617,359825,honey mustard cracker crumb chicken,this is a healthier version of fried chicken. ...
150669,161093,oven baked fish and chips,"from family circle, uk. a healthy version of ..."
168905,458924,quick double tomato chicken with artichoke hea...,"entered for safe-keeping, this is almost a ""cu..."
179112,56760,salmon potato boats,another recipe i've had for years from the bac...
